In [ ]:
import pandas as pd
import numpy as np
import evaluate

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    TrainingArguments, Trainer

In [ ]:
train = pd.read_csv("data/goodreads_train.csv")
train.head()

In [ ]:
data = train.loc[:, ["review_text", "rating"]]

In [ ]:
# create a validation set
train_df, valid_df = train_test_split(data, test_size=0.1, random_state=0)
print(train_df.shape, valid_df.shape)

In [ ]:
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)

In [ ]:
dataset = DatasetDict()
dataset["train"] = train_ds
dataset["valid"] = valid_ds

# target column should be named label
dataset = dataset.rename_columns({"rating": "label"})
dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_func(dataset, col="review_text"):
    return tokenizer(dataset[col], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_func, batched=True)
tokenized_datasets

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=6)                                                       

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions,
                          references=labels)

In [ ]:
training_args = TrainingArguments(output_dir="transformer_trainer",
                                 evaluation_strategy="epoch",
                                 per_device_train_batch_size=24,
                                 per_gpu_eval_batch_size=24)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()